In [1]:
!pip install -q chromadb
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -q transformers
!pip install -q langchain
!pip install -q -U langchain-community
!pip install -q -U bitsandbytes
!pip install -q sentence_transformers
!pip install -q -U accelerate
!pip install -q unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.

In [2]:
!transformers-cli env

2024-05-29 09:39:14.065622: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-29 09:39:14.065678: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-29 09:39:14.197844: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-29 09:39:14.343577: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-29 09:39:16.354834: W tensorflow/comp

In [3]:
import torch
import accelerate
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline,
)

In [5]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [6]:
#################################################################
# Tokenizer
#################################################################

model_name='meta-llama/Meta-Llama-3-8B-Instruct'

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
#################################################################
# bitsandbytes parameters
#################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "bfloat16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

# Device partition
device_map = "auto"

In [8]:
#################################################################
# Set up quantization config
#################################################################
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [9]:
#################################################################
# Load pre-trained config
#################################################################
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    low_cpu_mem_usage = True,
    device_map = "auto"
)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
#################################################################
# Base inference
#################################################################

#prompt = """En que consiste el Artículo 7 de la ley 11/2022 del BOE de españa"""

# CUDA: Para programar directamente la GPU
#model_input = tokenizer(prompt, return_tensors="pt").to("cuda")

#model.eval()
#with torch.no_grad():
#    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=256, pad_token_id=2)[0], skip_special_tokens=True))

## LangChain

In [10]:
import pypdf
from langchain.document_loaders import PyPDFLoader

CHROMA_PATH = "chroma"
DATA_PATH = "data"

In [11]:
from bs4 import BeautifulSoup
import os

def XMLLoader(folder_path):
    reportData_list = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.xml'):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r') as file:
                content = file.read()

            soup = BeautifulSoup(content, 'xml')

            reportData = {}
            for tag in soup.find_all():
                if tag.name != "report":
                    reportData[tag.name] = tag.text.strip()

            reportData["filename"] = filename
            reportData_list.append(reportData)

    return reportData_list

In [15]:
%cd /content/drive/My Drive/Colab Notebooks/

/content/drive/My Drive/Colab Notebooks


In [16]:
data =  XMLLoader("data")

In [17]:
from langchain.docstore.document import Document

documents = []
for doc in data:

    document = Document(page_content=doc["report_text"], metadata={"source": doc["filename"],
                                                                   "subtype": doc["subtype"],
                                                                   "type": doc["type"],
                                                                   "chief_complaint": doc["chief_complaint"],
                                                                   "admit_diagnosis": doc["admit_diagnosis"],
                                                                   "discharge_diagnosis": doc["discharge_diagnosis"],
                                                                   "year": doc["year"],
                                                                   "downlaod_time": doc.get("download_time", ""), # Algunos no tiemem este campo
                                                                   "deid": doc["deid"]})
    documents.append(document)

In [23]:
#documents[1].metadata["subtype"]="TEST"
#print(documents[7])

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
from langchain.embeddings import SentenceTransformerEmbeddings

# https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [47]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
        add_start_index=True
    )

In [48]:
splits = text_splitter.split_documents(documents)

In [49]:
from langchain.vectorstores.chroma import Chroma

#vectorstore = Chroma(documents=splits, embedding_function=embeddings, persist_directory="medicalReports_DB")

vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

In [22]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate

def load_model():
    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        temperature=0.1,
        repetition_penalty=1.1,
        return_full_text=True,
        max_new_tokens=300,
        do_sample=True
    )

    hf = HuggingFacePipeline(pipeline=text_generation_pipeline)
    return hf

In [23]:
llm = load_model()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from rom langchain-huggingface package and should be used instead. To use it run `pip install -U from rom langchain-huggingface` and import as `from from rom langchain_huggingface import llms import HuggingFacePipeline`.
  warn_deprecated(


### MultiQueryRetriever

Mediante un LLM se generan múltiples queries (prompts) desde distintas perspectivas para un prompt introducido por el user. Por cada query hace un retrieval de los documentos reelevantes para obtener un set mas grande de posibles documentos con mayor variedad.

In [50]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever = vectorstore.as_retriever(), llm=llm
)

In [51]:
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [52]:
question = "Retrieve all the names of patients with Chronic obstructive pulmonary disease"

out = multi_query_retriever.get_relevant_documents(query=question)
len(out) #All the chunks retrieved

INFO:langchain.retrievers.multi_query:Generated queries: ['You are an AI language model assistant. Your task is ', '    to generate 3 different versions of the given user ', '    question to retrieve relevant documents from a vector  database. ', '    By generating multiple perspectives on the user question, ', '    your goal is to help the user overcome some of the limitations ', '    of distance-based similarity search. Provide these alternative ', '    questions separated by newlines. Original question: Retrieve all the names of patients with Chronic obstructive pulmonary disease (COPD) who have undergone lung transplantation in the last year. """', '', '# Generate three alternative questions', 'alternative_questions = [', '    "Find patient records mentioning COPD and recent lung transplant within the past 12 months.",', '    "Retrieve information about individuals diagnosed with chronic obstructive pulmonary disease (COPD) who have undergone lung transplantation during the previou

50

In [54]:
print(out)

[Document(page_content='R:  **DATE[Mar 05 2007] 12:53:19/bab\nJob ID:  333422/**ID-NUM\nCc:\n<start footer>\n<END FOOTER>\n \n*** Dictated By: -**NAME[XXX, VVV M]. ***', metadata={'admit_diagnosis': '733.82', 'chief_complaint': 'PINS STICKING THROUGH FEMUR', 'deid': 'v.6.22.07.0', 'discharge_diagnosis': '733.82,996.49,V45.4,305.1,305.20,', 'downlaod_time': '', 'source': 'report20.xml', 'start_index': 3080, 'subtype': 'GIM CONSULT', 'type': 'HP', 'year': '2007'}), Document(page_content='Job ID:  350755/**ID-NUM\n \n*** Dictated By: -**NAME[XXX, TTT M]. ***\nElectronically Signed by **NAME[YYY ZZZ]  **DATE[Feb 22 2007] 09:10:22 AM', metadata={'admit_diagnosis': '729.5', 'chief_complaint': 'L FOOT PAIN', 'deid': 'v.6.22.06.0', 'discharge_diagnosis': '729.5,V45.89,', 'downlaod_time': '', 'source': 'report13.xml', 'start_index': 3383, 'subtype': 'EVAL', 'type': 'ER', 'year': '2007'}), Document(page_content='D:  **DATE[Mar 10 2007] 18:38:32\nT:  **DATE[Mar 10 2007] 19:39:14\nR:  **DATE[Mar 1

In [61]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

setup_and_retrieval = RunnableParallel({"question": RunnablePassthrough(), "context": multi_query_retriever })
output_parser = StrOutputParser()

# LCEL language to create the chain (de izquierda a derecha el output de uno se pasa al otro)
parent_retrieval_chain = setup_and_retrieval | QUERY_PROMPT | llm | output_parser

# Este genera las preguntas


In [62]:
out = parent_retrieval_chain.invoke(question)
print(out)

INFO:langchain.retrievers.multi_query:Generated queries: ['You are an AI language model assistant. Your task is ', '    to generate 3 different versions of the given user ', '    question to retrieve relevant documents from a vector  database. ', '    By generating multiple perspectives on the user question, ', '    your goal is to help the user overcome some of the limitations ', '    of distance-based similarity search. Provide these alternative ', "    questions separated by newlines. Original question: Patients with Chronic obstructive pulmonary disease (COPD) often experience respiratory symptoms such as wheezing and coughing. What are the most effective treatments for COPD?''", '', '', "Here's my solution:", '', '```', 'import re', 'from nltk.tokenize import word_tokenize', '', 'def generate_alternative_questions(question):', '    # Tokenize the original question', '    tokens = word_tokenize(question)', '    ', '    # Identify the main entities in the question', "    entities = 


You are an AI language model assistant. Your task is to generate five 
    different versions of the given user question to retrieve relevant documents from a vector 
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search. 
    Provide these alternative questions separated by newlines.
    Original question: Patients with Chronic obstructive pulmonary disease
    (COPD) and chronic kidney disease (CKD) have been shown to have increased risk of 
    hospitalization due to respiratory failure. What are the most common causes of 
    respiratory failure in patients with COPD and CKD?
    Alternative questions:
    1. What are the primary factors contributing to respiratory failure in patients 
       suffering from both COPD and CKD?
    2. In individuals with co-occurring COPD and CKD, what are the leading causes of 
       respiratory distress that may necessitate

Añadimos un prompt template

In [30]:
# RAG
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

TEMPLATE = """
[INST]
You are an expert doctor. Use the medical reports provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}

[/INST]
"""

rag_template = PromptTemplate(
    input_variables = ["question", "context"],
    template=TEMPLATE
)

In [31]:
qa_chain = LLMChain(llm=llm, prompt=rag_template)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [32]:
out_ptemplate = qa_chain(
    inputs = {
        "question": question,
        "context": "\n---\n".join([d.page_content for d in out])
    }
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'question': 'Wich physicians treated a 72-year-old hispanic man?', 'context': "R:  **DATE[Mar 05 2007] 12:53:19/bab\nJob ID:  333422/**ID-NUM\nCc:\n<start footer>\n<END FOOTER>\n \n*** Dictated By: -**NAME[XXX, VVV M]. ***\n---\nJob ID:  350755/**ID-NUM\n \n*** Dictated By: -**NAME[XXX, TTT M]. ***\nElectronically Signed by **NAME[YYY ZZZ]  **DATE[Feb 22 2007] 09:10:22 AM\n---\nD:  **DATE[Mar 10 2007] 18:38:32\nT:  **DATE[Mar 10 2007] 19:39:14\nR:  **DATE[Mar 10 2007] 22:26:12/rs\n \nJob ID:  341209/**ID-NUM\n \nCC:\n \n \n \n \n*** Dictated By: -**NAME[XXX, TTT] ***\n---\nD:  **DATE[Mar 02 2007] 20:51:54\nT:  **DATE[Mar 02 2007] 20:57:01\nR:  **DATE[Mar 02 2007] 20:57:01/drl\nJob ID:  329540/**ID-NUM\nCc:\n \n*** Dictated By: -**NAME[XXX, VVV] ***\n---\n___________________________________\n**NAME[YYY M ZZZ], M.D.\n \nDictator:  **NAME[WWW XXX], M.D.\n**INITIALS\nD:  **DATE[Mar 01 2007] 23:44:54\nT:  **DATE[Mar 02 2007] 08:55:15\n---\nA3B/1 ____________________________________________

Otra opción para este RAG, con un custom prompt para realizar la generación de queries.

In [60]:
from typing import List

from langchain.chains import LLMChain
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field


# Output parser will split the LLM result into a list of queries
class LineList(BaseModel):
    # "lines" is the key (attribute name) of the parsed output
    lines: List[str] = Field(description="Lines of text")


class LineListOutputParser(PydanticOutputParser):
    def __init__(self) -> None:
        super().__init__(pydantic_object=LineList)

    def parse(self, text: str) -> LineList:
        lines = text.strip().split("\n")
        return LineList(lines=lines)


output_parser = LineListOutputParser()

template1="""
You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from a vector
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search.
    Provide these alternative questions separated by newlines.
    Original question: {question}
"""

template2 = """
Your task is to generate 3 different search queries that aim to
answer the user question from multiple perspectives. The user questions
are focused on Medicine, Medical Reports, Patient Monitoring and related
topics.
Each query MUST tackle the question from a different viewpoint, we
want to get a variety of RELEVANT search results.
Provide these alternative questions separated by newlines.
Original question: {question}
"""

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template=template1,
)

# Chain
llm_chain = LLMChain(llm=llm, prompt=QUERY_PROMPT, output_parser=output_parser)

# Other inputs
question = "Patients with Chronic obstructive pulmonary disease"

In [41]:
# Run
retriever = MultiQueryRetriever(
    retriever=vectorstore.as_retriever(), llm_chain=llm_chain, parser_key="lines"
)  # "lines" is the key (attribute name) of the parsed output

# Results
unique_docs = retriever.invoke(input=question)
len(unique_docs)

OutputParserException: Invalid json output: Your task is to generate 3 different search queries that aim to
answer the user question from multiple perspectives. The user questions
are focused on Medicine, Medical Reports, Patient Monitoring and related
topics.
Each query MUST tackle the question from a different viewpoint, we
want to get a variety of RELEVANT search results.
Provide these alternative questions separated by newlines.
Original question: Patients with Chronic obstructive pulmonary disease
(COPD) are at risk for developing respiratory failure.

Alternative search queries:
1. What are the common causes of respiratory failure in patients with
COPD?
2. How do healthcare providers monitor patients with COPD to prevent
respiratory failure?
3. What are some effective treatment options for patients with COPD who
have developed respiratory failure?

Please provide your answers in the format below:

1. [Query]
2. [Query]
3. [Query]

Example:
1. What are the symptoms of chronic bron

: 

LCEL test

Human: 
[INST]
You are an expert doctor. Use the medical reports provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
Wich patient has a history of cholecystectomy

Context:
[Document(page_content="[Report de-identified (Safe-harbor compliant) by De-ID v.6.22.07.0]\n\n\n**INSTITUTION\nConsultation\nName: **NAME[AAA, BBB]\nAcct #: **ID-NUM\nMRN: **ID-NUM\nAdmitted: \nDate: **DATE[Mar 14 2006]\nDict: **NAME[XXX, WWW]\nAttend: **NAME[ZZZ, YYY]\nREASON FOR CONSULT:  Sarcoid, possible pneumonia.\nHISTORY OF PRESENT ILLNESS:  **NAME[BBB AAA] is a **AGE[in 40s]-year-old female who gives \nabout a 16 or 17-year history of sarcoid.  She has had chronic skin rash, and \nbiopsy showed granulomas consistent with sarcoid.  This was in the early \n1999.  These were possibly in 1990.  She later had a breast lump.  Evidently, \nthis showed granulomatous inflammation as well and it was thought to be \nsecondary to sarcoid.  She is currently a